Learning Agents and Utility-Based Agents Using PEAS

Case Study Overview
   
The system is a smart fertilizer management agent used in a farm.
It recommends the most suitable fertilizer type for crops based on:

    Soil nutrient level and moisture
    Weather conditions
    Crop growth stage
    Cost of fertilizer


PEAS ANALYSIS

    Performance:
        - maximize crop yield
        - minimize cost
        - maintain soil sustainability
    
    Environment:
        - farm field
        - soil nutrient level
        - weather
        - crop growth stage
    
    Actuators:
        - apply fertilizer type
    
    Sensors:
        - soil sensors
        - moisture sensors
        - weather monitoring
        - crop growth monitoring

Environment state representation.

In [2]:
#environment state class
class FarmState: #created a class called 'FarmState' (:) => start of the class bosy
    def __init__(self, soil_n,moisture,weather,growth_stage): #variables => parameters
        #(def) => defining a function (__init__) new object is created, function not creating a variable, rather assigning variable
        self.soil_n = soil_n  #(self) =>specific object os created (has own data)
        self.moisture= moisture
        self.weather = weather #e.g storing weather isnide object (self.weather)
        self.growth_stage = growth_stage 

In [3]:
#Fertilizer Knowledge Base
fertilizers = {
    "organic": {"yield": 6, "cost": 5, "sustain": 9},   #relative scores
    "urea": {"yield": 9, "cost": 8, "sustain": 4},
    "compost": {"yield": 5, "cost": 4, "sustain": 10}
}
#scale 1- very low, 10 - very high

Environment State Categories

    Soil nutrient level: low, medium, high
    Moisture level: low, medium, high
    Weather condition: sunny, cloudy, rainy
    Growth stage: early, mid, late

In [4]:
#no need to match nnames with class farmstate
soil_effect = {"low": 0.5, "medium": 0, "high": -0.3} #soil nutrient level
moisture_effect = {"low": -0.3, "medium": 0, "high": 0.2}
weather_effect = {"sunny": 0.1, "cloudy": 0, "rainy": -0.2}
stage_effect = {"early": 0.3, "mid": 0, "late": -0.2}

#affects attribute Yield(X) => relative values only

In [5]:
#state --> Yield Modifier
def state_modifier(state):
    modifier = 1   #baseline 
    #X = fert["yield"] * state_modifier(state)

    modifier += soil_effect.get(state.soil_n, 0) #=> 0 means if not found in dictionary = no chnages 1*0 = 0 
    modifier += moisture_effect.get(state.moisture, 0) #e.g the agent detected "high" moisture =>modifier = 1 + 0.2 = 1.2
    modifier += weather_effect.get(state.weather, 0)
    modifier += stage_effect.get(state.growth_stage, 0)

    return modifier
#note: how it affects yield
#X = base_yield × modifier
#9 × 1.2 = 10.8 (9 is from fert knowlege base 1.2 is the mod baseline * moisture (high)
#yield increase 20% => increase ÷ original => 1.8 ÷ 9 = 0.2 => 20%

In [6]:
#Utility Function
def compute_utility(fert, state): #parameters are placeholders (state)=> independent from the above code, we can rename it doesnt mattter
    #but if we remove it, we will get wrong stuff since we expected 2 inputs fort (X)
    X = fert["yield"] * state_modifier(state)
    Y = fert["cost"]
    Z = fert["sustain"]

    U = 5*X - 3*Y + 2*Z
    return U
#Note: we got the values from the knowledge base
#but variable yield (X) is affected with environments

In [7]:
#Agent Decision
def choose_fertilizer(state):
    best_action = None #reserve keyword like 'null' in java
    best_utility = -999 #baseline => start with a value that is guaranteed to be smaller than any real utility

    for name, fert in fertilizers.items():
        u = compute_utility(fert, state) #calculate => how good is this fertilizer in this state

        if u > best_utility: #check  if current u is better than one it seens before
            best_utility = u
            best_action = name

    return best_action, best_utility
#note: best_action => highest utility option (fert name) / best_utility => utility score

In [8]:
#Learning Agent Component
experience = {} #store past outcomes for each action (fertilizer) => agent memory

def update_learning(action, reward): #action=> (fert name)/reward=>performance score
    experience[action] = experience.get(action, 0) + reward #if action not recorded =return 0
    
#Note (rewards)
#no "urea" yet=>default 0
#0 + 8 = 8 = store
#significance: The code allows the agent to record past performance of fertilizer actions,
#showing learning capability but...this experience is not yet used to modify decisions

In [9]:
# Scenario 1
state = FarmState("low", "medium", "sunny", "early")

print("Scenario 1: low soil, medium moisture, sunny weather, early stage\n")
print("Fertilizer\tX\tY\tZ\tUtility")

for name, fert in fertilizers.items():
    X = fert["yield"] * state_modifier(state)
    Y = fert["cost"]
    Z = fert["sustain"]
    U = compute_utility(fert, state)

  
    print(f"{name:<15}{X:<9.2f}{Y:<8}{Z:<8}{U:<8.2f}") #(<) left-shift spacing alignment

best_action, best_utility = choose_fertilizer(state) # evaluate> fertilizer name> utilty score :: purpose=> computation

print("\nDecision:", best_action)
print("Reason: Highest utility value.")

Scenario 1: low soil, medium moisture, sunny weather, early stage

Fertilizer	X	Y	Z	Utility
organic        11.40    5       9       60.00   
urea           17.10    8       4       69.50   
compost        9.50     4       10      55.50   

Decision: urea
Reason: Highest utility value.


In [12]:
# Scenario 2
state = FarmState("low", "medium", "rainy", "late")

print("Scenario 2: low soil, medium moisture, rainy weather, late stage\n")
print("Fertilizer\tX\tY\tZ\tUtility")

for name, fert in fertilizers.items():
    X = fert["yield"] * state_modifier(state)
    Y = fert["cost"]
    Z = fert["sustain"]
    U = compute_utility(fert, state)

    #print(f"{name}\t{X:.2f}\t{Y}\t{Z}\t{U:.2f}")
    print(f"{name:<15}{X:<9.2f}{Y:<8}{Z:<8}{U:<8.2f}") #(<) left-shift spacing alignment

best_action, best_utility = choose_fertilizer(state)

print("\nDecision:", best_action)
print("Reason: Highest utility value.")

Scenario 2: low soil, medium moisture, rainy weather, late stage

Fertilizer	X	Y	Z	Utility
organic        6.60     5       9       36.00   
urea           9.90     8       4       33.50   
compost        5.50     4       10      35.50   

Decision: organic
Reason: Highest utility value.


In [13]:
# Scenario 3
state = FarmState("high", "high", "sunny", "late")

print("Scenario 3: high soil, high moisture, sunny weather, late stage\n")
print("Fertilizer\tX\tY\tZ\tUtility")

for name, fert in fertilizers.items():
    X = fert["yield"] * state_modifier(state)
    Y = fert["cost"]
    Z = fert["sustain"]
    U = compute_utility(fert, state)

    #print(f"{name}\t{X:.2f}\t{Y}\t{Z}\t{U:.2f}")
    print(f"{name:<15}{X:<9.2f}{Y:<8}{Z:<8}{U:<8.2f}") #(<) left-shift spacing alignment

best_action, best_utility = choose_fertilizer(state)

print("\nDecision:", best_action)
print("Reason: Highest utility value.")

Scenario 3: high soil, high moisture, sunny weather, late stage

Fertilizer	X	Y	Z	Utility
organic        4.80     5       9       27.00   
urea           7.20     8       4       20.00   
compost        4.00     4       10      28.00   

Decision: compost
Reason: Highest utility value.
